# Lab: SQL and Database

This lab focuses on extracting and displaying information from a database. 

Please run all the code chunks until you see the lab exercises (at the very bottom). 

In [10]:
# import libraries
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, and_, or_
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy import func

# Setting things up:
engine = sqlalchemy.create_engine('sqlite:///geog.db')
Base = declarative_base() 

# Schemas
class Region(Base):
  __tablename__ = 'regions'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  departments = relationship("Department", backref = "region")

  def __init__(self, name):
    self.name = name 

  def __repr__(self):
    return "<Region('%s')>" % self.id 

class Department(Base):
  __tablename__ = 'departments'

  id = Column(Integer, primary_key=True)
  deptname = Column(String)
  region_id = Column(Integer, ForeignKey('regions.id')) 
  towns = relationship("Town", backref = "department")

  def __init__(self, deptname):
    self.deptname = deptname 

  def __repr__(self):
    return "<Department('%s')>" % self.id 

class Town(Base):
  __tablename__ = 'towns'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  population = Column(Integer)
  dept_id = Column(Integer, ForeignKey('departments.id'))

  def __init__(self, name, population):
    self.name = name 
    self.population = population

  def __repr__(self):
    return "<Town('%s')>" % (self.name)

#First time create tables
Base.metadata.create_all(engine) 

#Create a session to actually store things in the db
Session = sessionmaker(bind=engine)
session = Session()

# Create regions
reg1 = Region('Region 1')
reg2 = Region('Region 2')
reg3 = Region('Region 3')

# Create departments, nested in regions
dept1 = Department('Department 1')
reg1.departments.append(dept1)

dept2 = Department('Department 2')
reg1.departments.append(dept2)

dept3 = Department('Department 3')
reg3.departments.append(dept3)

dept4 = Department('Department 4')
reg2.departments.append(dept4)


# Create towns, nested in departments
t1 = Town("a", 10000)
t2 = Town("b", 20000)
dept1.towns = [t1, t2]

t3 = Town("c", 30000)
t4 = Town("d", 40000)
dept2.towns = [t3, t4]

t5 = Town("e", 50000)
t6 = Town("f", 60000)
dept3.towns = [t5, t6]

t7 = Town("g", 70000)
t8 = Town("h", 80000)
dept4.towns = [t7, t8]

session.add_all([reg1, reg2, reg3])
session.add_all([dept1, dept2, dept3, dept4])
session.add_all([t1, t2, t3, t4, t5, t6, t7, t8])

session.commit()

/var/folders/rc/6my2lrc536j9hknp62n5hbsm0000gp/T/ipykernel_77260/4260505936.py:10: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


1. Print all of the towns, and order them by Town.id in descending order.

In [15]:
towns = session.query(Town).order_by(Town.id.desc()).all()

for town in towns:
    print(f"Town(id={town.id}, name='{town.name}', population={town.population})")


Town(id=56, name='f', population=60000)
Town(id=55, name='e', population=50000)
Town(id=54, name='h', population=80000)
Town(id=53, name='g', population=70000)
Town(id=52, name='d', population=40000)
Town(id=51, name='c', population=30000)
Town(id=50, name='b', population=20000)
Town(id=49, name='a', population=10000)
Town(id=48, name='f', population=60000)
Town(id=47, name='e', population=50000)
Town(id=46, name='h', population=80000)
Town(id=45, name='g', population=70000)
Town(id=44, name='d', population=40000)
Town(id=43, name='c', population=30000)
Town(id=42, name='b', population=20000)
Town(id=41, name='a', population=10000)
Town(id=40, name='f', population=60000)
Town(id=39, name='e', population=50000)
Town(id=38, name='h', population=80000)
Town(id=37, name='g', population=70000)
Town(id=36, name='d', population=40000)
Town(id=35, name='c', population=30000)
Town(id=34, name='b', population=20000)
Town(id=33, name='a', population=10000)
Town(id=32, name='f', population=60000)


2. Display, by department, the cities having more than 50,000 inhabitants.

In [18]:
departments = session.query(Department).join(Town).filter(Town.population > 50000).all()

for department in departments:
    print(f"Department: {department.deptname}")
    for town in department.towns:
        if town.population > 50000:
            print(f"  Town: {town.name}, Population: {town.population}")


Department: Department 4
  Town: g, Population: 70000
  Town: h, Population: 80000
Department: Department 3
  Town: f, Population: 60000
Department: Department 4
  Town: g, Population: 70000
  Town: h, Population: 80000
Department: Department 3
  Town: f, Population: 60000
Department: Department 4
  Town: g, Population: 70000
  Town: h, Population: 80000
Department: Department 3
  Town: f, Population: 60000


3. Display the total number of inhabitants per department using only a query. (no lists!)

In [19]:
from sqlalchemy import func

result = session.query(
    Department.deptname, 
    func.sum(Town.population).label('total_population')
).join(Town).group_by(Department.id).all()

for deptname, total_population in result:
    print(f"Department: {deptname}, Total Population: {total_population}")


Department: Department 1, Total Population: 30000
Department: Department 2, Total Population: 70000
Department: Department 4, Total Population: 150000
Department: Department 3, Total Population: 110000
Department: Department 1, Total Population: 30000
Department: Department 2, Total Population: 70000
Department: Department 4, Total Population: 150000
Department: Department 3, Total Population: 110000
Department: Department 1, Total Population: 30000
Department: Department 2, Total Population: 70000
Department: Department 4, Total Population: 150000
Department: Department 3, Total Population: 110000
